In [12]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import torch
import math
from torch.utils.tensorboard import SummaryWriter # type: ignore

from agents.A2C_cliff import A2CAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from torch.nn import functional as F
from collections import defaultdict
import itertools
from stable_baselines3 import A2C
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# env = gym.make("CliffWalking-v0")
# # env = GridWorldEnv(size=3,fixed_map = True, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# model = A2C("MlpPolicy", env, tensorboard_log="./runs/", verbose=1)
# # model = A2C("MultiInputPolicy", env, tensorboard_log="./runs/", verbose=1)
# model.learn(total_timesteps=10_0000) # 所以训练无 fobidden 的地图需要 10_0000 次 (反正 1_0000 是不够的) 
# env.close()

In [14]:
# demo_env = gym.make("CliffWalking-v0", render_mode="human")
# observation, _  = demo_env.reset()
# for _ in range(1000):
#     # observation = (observation,)
#     action = model.predict(observation)  # agent policy that uses the observation and info
#     # insert an algorithm that can interact with env and output an action here
#     observation, reward, terminated, truncated, info = demo_env.step(int(action[0]))
#     if terminated or truncated:
#         observation, info = demo_env.reset()
#         # print(steps)

In [15]:
LEARN_RATE = 1e-2
DISCOUNTED_FACTOR = 0.99

FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 1

SEED = 666

In [16]:
# env = GridWorldEnv(size=3,fixed_map = True, seed=SEED, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)

# env = gym.make("CliffWalking-v0")
# torch.autograd.set_detect_anomaly(False, check_nan=True)
# env.seed(args.seed)
torch.manual_seed(SEED)

In [17]:
episode_rewards = []
episode_lengths = []



Cliff Walking

    0: Move up

    1: Move right

    2: Move down

    3: Move left


In [18]:
env = gym.make("CliffWalking-v0")
agent = A2CAgent(int(env.observation_space.n), int(env.action_space.n),  discounted_factor=0.99999)

In [19]:

CLIFF_STATE = 37

# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# agent = PGAgent(2, 5, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)

# env = gym.make("CartPole-v1")
# agent = PGAgent(4, 2, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)

writer = SummaryWriter()
num_episodes = 100000
episode_len = 50000
trajectory = []
obs, _ = env.reset()
episode_rewards = defaultdict(float)
TD_step = 5 # perform TD(5)
for i_episode  in range(num_episodes):
    # 首先, 根据 policy 生成 episode
    terminated = truncated = False
    next_obs, _ = env.reset()
    episode_record = []
    # del agent.saved_log_probs[:]

    for _ in range(TD_step-1):
        # del agent.saved_log_prob
        obs = next_obs
        action = agent.get_action(obs) # action 这里也有随机性
        next_obs, reward, terminated, truncated, info = env.step(action)
        if reward == -100:
            next_obs = CLIFF_STATE 
            truncated = True
        episode_record.append((obs, action, reward, next_obs))
        
        if terminated or truncated:
            break

    for t in range(episode_len):
        if not (terminated or truncated):
            obs = next_obs # 注意这里的 state 要和 之前环境里的 agent 的位置对上 (TD queue中最后一个), 不要被下面计算用的 state 干扰
            action = agent.get_action(obs) # action 这里也有随机性
            next_obs, reward, terminated, truncated, info = env.step(action)
            if reward == -100:
                next_obs = CLIFF_STATE 
                truncated = True
            episode_record.append((obs, action, reward, next_obs))

        if len(episode_record) == 0:
            break

        if not (terminated or truncated):
            # with torch.no_grad():
            discounted_reward = agent.value_net(next_obs) 
        else: # next state is the end, although maybe I should still use v estimate
            discounted_reward = torch.tensor(0.)
        for i, (state, action, reward, next_state) in enumerate(episode_record[::-1]):
            discounted_reward = reward + agent.discounted_factor * discounted_reward

        # discounted_reward = sum(agent.discounted_factor**i * t[2] for i, t in enumerate(episode_record[t:]))
        state, action, reward, next_state = episode_record.pop(0)

        # total_return = sum(agent.discounted_factor**i * t.reward for i, t in enumerate(episode_record[t:]))

        # value_next = agent.value_net(next_state)
        # with torch.no_grad():
        v_value = agent.value_net(state).squeeze() 

        # TD_target = reward + agent.discounted_factor * value_next
        TD_target = discounted_reward.squeeze() 
        TD_error = TD_target - v_value # TD_error is the advantage (每一处加减, 都要注意 broadcasting)

        # Update statistics
        episode_rewards[i_episode] += reward
        # with torch.no_grad():
        action_probs = agent.policy_net(state)
        action_prob = action_probs[0][action]

        agent.optimizer_v.zero_grad()
        loss2 =  F.mse_loss(TD_target, v_value)
        loss2.sum().backward(retain_graph=True)
        agent.optimizer_v.step()

        agent.optimizer.zero_grad()
        # loss = -agent.saved_log_prob * TD_error
        loss = -action_prob * TD_error
        loss.sum().backward()
        # torch.nn.utils.clip_grad.clip_grad_norm_(agent.policy_net.parameters(), 100)
        agent.optimizer.step()
        if t % 1000 == 0:
            print("\rStep {} @ Episode {}/{} ({}, {})".format(
                    t, i_episode + 1, num_episodes, episode_rewards[i_episode], episode_rewards[i_episode - 1]))

        # if terminated or truncated:
        #     break
print("end train")

Step 0 @ Episode 1/100000 (-100.0, 0.0)
Step 0 @ Episode 2/100000 (-100.0, -100.0)
Step 0 @ Episode 3/100000 (-1.0, -100.0)
Step 0 @ Episode 4/100000 (-1.0, -104.0)
Step 0 @ Episode 5/100000 (-1.0, -103.0)
Step 0 @ Episode 6/100000 (-100.0, -121.0)
Step 0 @ Episode 7/100000 (-1.0, -100.0)
Step 0 @ Episode 8/100000 (-1.0, -102.0)
Step 0 @ Episode 9/100000 (-1.0, -102.0)
Step 0 @ Episode 10/100000 (-100.0, -112.0)
Step 0 @ Episode 11/100000 (-1.0, -100.0)
Step 0 @ Episode 12/100000 (-1.0, -105.0)
Step 0 @ Episode 13/100000 (-1.0, -112.0)
Step 0 @ Episode 14/100000 (-1.0, -103.0)
Step 0 @ Episode 15/100000 (-100.0, -136.0)
Step 0 @ Episode 16/100000 (-1.0, -100.0)
Step 0 @ Episode 17/100000 (-1.0, -106.0)
Step 0 @ Episode 18/100000 (-1.0, -107.0)
Step 0 @ Episode 19/100000 (-100.0, -101.0)
Step 0 @ Episode 20/100000 (-1.0, -100.0)
Step 0 @ Episode 21/100000 (-1.0, -120.0)
Step 0 @ Episode 22/100000 (-100.0, -101.0)
Step 0 @ Episode 23/100000 (-100.0, -100.0)
Step 0 @ Episode 24/100000 (-1

KeyboardInterrupt: 

cliffwalk 大约 1000 步左右, 开始有得解迹象

TD5 将近 15000 步, 完全收敛到最优 (3000 左右接近收敛, 但是10000步收敛到了次优, )

In [24]:
action_mappings = ["↑","→","↓","←"]
for i in range(48):
    if i % 12 == 0:
        print()
    action = agent.get_action(i)
    print(action_mappings[action], end=" ")


→ → → → → → → → → → → ↓ 
→ → → → → → → → → → → ← 
→ → → → → → → → → → → ↓ 
↑ → → → → → → → → → → → 

In [25]:
torch.save(agent.policy_net.state_dict(), "model_saves/A2C_cliffWalk_optimal_policy")
torch.save(agent.value_net.state_dict(), "model_saves/A2C_cliffWalk_optimal_v")

In [ ]:
# # visualize_in_gym(agent, "CartPole-v1")
# policy = agent.generate_policy_table(env.height, env.width)

# print_by_dict(env, policy)

# for i in range(env.height):
#     print("[", end=" ")
#     for j in range(env.width):
#         state = (i,j)
#         action = np.argmax(policy[state])
#         print(env.action_mappings[action], end=" ")
#     print("]")

In [26]:
# env.max_steps = 10
# gridworld_demo(agent, env, repeat_times=500)
# gridworld_demo(agent, forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# visualize_in_gym(agent, "CartPole-v1")
visualize_in_gym(agent, "CliffWalking-v0", steps=100)


12
25
38
51
64
77
90
